# First look at computing derivatives analytically
---
In this notebook we try to test the features of sympy automatic derivative calculator. Given an energy function, we want to compute the following:
- The force
$$\vec{F} = - \nabla E$$
- Force jacobians for implicit euler integration:
$$\frac{\partial \vec{F}}{\partial \vec{x}}\quad\quad\frac{\partial \vec{F}}{\partial \vec{v}}$$
- The derivative of the force wrt the phisical parameters (for differenciable simulations)
$$\frac{\partial \vec{F}}{\partial \vec{p}}$$

In [1]:
import sympy as sym

x1, x2, y1, y2, z1, z2 = sym.symbols("x1 x2 y1 y2 z1 z2")
p1 = sym.Matrix([x1, y1, z1])
p2 = sym.Matrix([x2, y2, z2])
L2 = (p1 - p2).dot(p1-p2)
L = sym.Symbol("L", positive=True)

energy = sym.Rational(1,2) * sym.Symbol("k") * ( L - sym.Symbol("L0"))**2
denergy = energy
energy = energy.subs(L, sym.sqrt(L2))
denergy

k*(L - L0)**2/2

In [2]:
force1 = [sym.diff(energy, sym.Symbol(l)) for l in ["x1", "y1", "z1"]]
force2 = [sym.diff(energy, sym.Symbol(l)) for l in ["x2", "y2", "z2"]]

vforce1 = sym.Matrix(force1)
vforce1 = sym.simplify(vforce1.subs(L2, L**2))
vforce1

Matrix([
[k*(L - L0)*(x1 - x2)/L],
[k*(L - L0)*(y1 - y2)/L],
[k*(L - L0)*(z1 - z2)/L]])

In [3]:
dfdx1 = []
for f1 in force1:
    dfdx1.append([])
    for l1 in [x1, y1, z1]:
        dfdx1[-1].append(sym.diff(f1, l1))

In [4]:
test = sym.simplify(sym.Matrix(dfdx1).subs(L2, L**2))
test

Matrix([
[k - L0*k/L + L0*k*(x1 - x2)**2/L**3,       L0*k*(x1 - x2)*(y1 - y2)/L**3,       L0*k*(x1 - x2)*(z1 - z2)/L**3],
[      L0*k*(x1 - x2)*(y1 - y2)/L**3, k - L0*k/L + L0*k*(y1 - y2)**2/L**3,       L0*k*(y1 - y2)*(z1 - z2)/L**3],
[      L0*k*(x1 - x2)*(z1 - z2)/L**3,       L0*k*(y1 - y2)*(z1 - z2)/L**3, k - L0*k/L + L0*k*(z1 - z2)**2/L**3]])

In [5]:
from sympy.codegen.cutils import render_as_source_file
render_as_source_file(test.tolist())

'\n\n{{k - L0*k/L + L0*k*pow(x1 - x2, 2)/pow(L, 3), L0*k*(x1 - x2)*(y1 - y2)/pow(L, 3), L0*k*(x1 - x2)*(z1 - z2)/pow(L, 3)}, {L0*k*(x1 - x2)*(y1 - y2)/pow(L, 3), k - L0*k/L + L0*k*pow(y1 - y2, 2)/pow(L, 3), L0*k*(y1 - y2)*(z1 - z2)/pow(L, 3)}, {L0*k*(x1 - x2)*(z1 - z2)/pow(L, 3), L0*k*(y1 - y2)*(z1 - z2)/pow(L, 3), k - L0*k/L + L0*k*pow(z1 - z2, 2)/pow(L, 3)}}'

In [6]:
dfdp = []
for f1 in force1:
    dfdp.append([])
    for l1 in ["k", "L0"]:
        dfdp[-1].append(sym.diff(f1, l1))

In [7]:
test2 = sym.simplify(sym.Matrix(dfdp).subs(L2, L**2))
test2

Matrix([
[(L - L0)*(x1 - x2)/L, k*(-x1 + x2)/L],
[(L - L0)*(y1 - y2)/L, k*(-y1 + y2)/L],
[(L - L0)*(z1 - z2)/L, k*(-z1 + z2)/L]])

In [8]:
render_as_source_file(test2.tolist())

'\n\n{{(L - L0)*(x1 - x2)/L, k*(-x1 + x2)/L}, {(L - L0)*(y1 - y2)/L, k*(-y1 + y2)/L}, {(L - L0)*(z1 - z2)/L, k*(-z1 + z2)/L}}'

In [9]:
keyword = "@PARAMETER"
substitute = render_as_source_file(test.tolist())
with open("test.cpp", "r") as sourcefile:
    read_content = sourcefile.read()
    iind = read_content.find(keyword)
    find = iind + len(keyword)
    new_content = read_content[0:iind] + substitute + read_content[find:]
    generatedfile = open("output.cpp", "w")
    generatedfile.write(new_content)
    generatedfile.close()

In [10]:
def substitute_words(keyword : str, substitute : str, filecontent : str) -> str:
    start_index = filecontent.find(keyword)
    if (start_index < 0): return filecontent
    end_index = start_index + len(keyword)
    new_content = filecontent[0:start_index] + substitute + filecontent[end_index:]
    return substitute_words(keyword, substitute, new_content)

In [11]:
import sympy as sym
x1, x2, y1, y2, z1, z2 = sym.symbols("x1 x2 y1 y2 z1 z2")
vx1, vx2, vy1, vy2, vz1, vz2 = sym.symbols("vx1 vx2 vy1 vy2 vz1 vz2")
p1 = sym.Matrix([x1, y1, z1])
p2 = sym.Matrix([x2, y2, z2])
L2 = (p1 - p2).dot(p1-p2)
L = sym.Symbol("L", positive=True)

energy = sym.Rational(1,2) * sym.Symbol("k") * ( L - sym.Symbol("L0"))**2
energy = energy.subs(L, sym.sqrt(L2))

parameters = [sym.Symbol("k"), sym.Symbol("L0")]

differenciable_parameters = [sym.Symbol("k")]

class InteractionGenerator:
    def __init__(self, energy, parameters, differenciable_parameters, classname):
        self.energy = energy
        self.parameters = parameters
        self.differenciable_parameters = differenciable_parameters
        self.classname = classname
        
        self.energy_str = sym.ccode(sym.simplify(energy.subs(L2, L**2)))
        self.parameters_array = sym.ccode([p for p in parameters])
        
        self.calculate_force()
        self.calculate_force_position_derivative()
        self.calculate_force_velocity_derivative()
        self.calculate_force_parameter_derivative()
    
    def calculate_force(self):
        self.force = [-sym.diff(energy, sym.Symbol(l)) for l in ["x1", "y1", "z1"]]
        
        # Force string
        tmp = sym.Matrix(self.force)
        tmp = sym.simplify(tmp.subs(L2, L**2))
        self.force_str = sym.ccode(tmp.tolist())
    
    def print_force(self):
        tmp = sym.Matrix(self.force)
        tmp = sym.simplify(tmp.subs(L2, L**2))
        return(tmp)
    
    def calculate_force_position_derivative(self):
        self.dfdx = []
        for f in self.force:
            self.dfdx.append([])
            for l1 in (x1, y1, z1):
                self.dfdx[-1].append(sym.diff(f, l1))
        
        tmp = sym.simplify(sym.Matrix(self.dfdx).subs(L2, L**2))
        self.dfdx_str = sym.ccode(tmp.tolist())
        
    def calculate_force_velocity_derivative(self):
        dfdv = []
        for f in self.force:
            dfdv.append([])
            for l1 in [vx1, vy1, vz1]:
                dfdv[-1].append(sym.diff(f, l1))
        self.dfdv = dfdv
        
        tmp = sym.simplify(sym.Matrix(self.dfdv).subs(L2, L**2))
        self.dfdv_str = sym.ccode(tmp.tolist())
        
    def print_dfdx(self):
        tmp = sym.simplify(sym.Matrix(self.dfdx).subs(L2, L**2))
        return tmp
    
    def print_dfdv(self):
        tmp = sym.simplify(sym.Matrix(self.dfdv).subs(L2, L**2))
        return tmp
    
    def calculate_force_parameter_derivative(self):
        dfdp = []
        for f in self.force:
            dfdp.append([])
            for l1 in differenciable_parameters:
                dfdp[-1].append(sym.diff(f, l1))
                
        self.dfdp = dfdp
        
        tmp = sym.simplify(sym.Matrix(self.dfdp).subs(L2, L**2))
        self.dfdp_str = sym.ccode(tmp.tolist())
    
    def print_dfdp(self):
        tmp = sym.simplify(sym.Matrix(self.dfdp).subs(L2, L**2))
        return tmp
    
    def write_file(self, readfilepath : str, writefilepath : str):
        for i, p in enumerate(self.parameters):
            self.energy_str = substitute_words(p.name, f"parameters[{i}]", self.energy_str)
            self.force_str = substitute_words(p.name, f"parameters[{i}]", self.force_str)
            self.dfdx_str = substitute_words(p.name, f"parameters[{i}]", self.dfdx_str)
            self.dfdv_str = substitute_words(p.name, f"parameters[{i}]", self.dfdv_str)
            self.dfdp_str = substitute_words(p.name, f"parameters[{i}]", self.dfdp_str)
        
        ClassToGenerate = {
            "@CLASSNAME" : self.classname,
            "@WARNING_TEXT" : "\n!!!!!!!!!!!!!!!!!!!\nDO NOT EDIT THIS FILE\n!!!!!!!!!!!!!!!!!!!\nThis file has been generated from the template <python_interaction.cpp> using a python script.\n",
            "@NUMBER_OF_PARAMETERS" : len(self.parameters),
            "@PARAMETERS_ARRAY" : self.parameters_array,
            "@ENERGY_CALCULATION" : self.energy_str,
            "@FORCE_CALCULATION" : self.force_str,
            "@FORCE_POSITION_DERIVATIVE_CALCULATION" : self.dfdx_str,
            "@FORCE_VELOCITY_DERIVATIVE_CALCULATION" : self.dfdv_str,
            "@FORCE_PARAMETERS_DERIVATIVE_CALCULATION" : self.dfdp_str,
        }
        with open(readfilepath, "r") as readfile:
            content = readfile.read()
            for keyword, value in zip(ClassToGenerate.keys(), ClassToGenerate.values()):
                content = substitute_words(keyword, str(value), content)
                with open(writefilepath, 'w') as writefile:
                    writefile.write(content)

In [12]:
a = InteractionGenerator(energy, parameters, differenciable_parameters,"TestingSpring")
a.print_force()
#sym.simplify(sym.diff(a.force[0],z1).subs(L2, L**2))

Matrix([
[-k*(L - L0)*(x1 - x2)/L],
[-k*(L - L0)*(y1 - y2)/L],
[-k*(L - L0)*(z1 - z2)/L]])

In [13]:
a.print_dfdx()

Matrix([
[-k + L0*k/L - L0*k*(x1 - x2)**2/L**3,       -L0*k*(x1 - x2)*(y1 - y2)/L**3,       -L0*k*(x1 - x2)*(z1 - z2)/L**3],
[      -L0*k*(x1 - x2)*(y1 - y2)/L**3, -k + L0*k/L - L0*k*(y1 - y2)**2/L**3,       -L0*k*(y1 - y2)*(z1 - z2)/L**3],
[      -L0*k*(x1 - x2)*(z1 - z2)/L**3,       -L0*k*(y1 - y2)*(z1 - z2)/L**3, -k + L0*k/L - L0*k*(z1 - z2)**2/L**3]])

In [15]:
a.write_file("../backward_euler/include/python_interaction.hpp",
            "../backward_euler/include/spring_test.hpp")